### Tools
Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:
1. A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
2. A function or coroutine to execute.

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("Why do parrots talk?")
response

AIMessage(content='<think>\nOkay, so I need to figure out why parrots talk. Let me start by what I know. Parrots are known for their ability to mimic human speech. But why do they do that? I remember reading that some birds can imitate sounds, but why specifically parrots? \n\nFirst, maybe it\'s for communication. In the wild, do parrots use vocalizations to interact with each other? I think they have various calls for different purposes, like warning of danger or calling mates. So perhaps when they talk in captivity, it\'s an extension of their natural communication behavior.\n\nAnother thought: social bonding. Parrots are social animals. In the wild, they live in flocks. Maybe they mimic sounds to bond with their flock members. If they\'re kept as pets, they might try to bond with their owners by imitating their speech. That makes sense because they see their owners as part of their flock.\n\nAlso, there\'s the aspect of reinforcement. If a parrot hears a word or phrase and gets a po

In [5]:
from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"


model_with_tools=model.bind_tools([get_weather])

In [6]:
response = model_with_tools.invoke("What's the weather like in Boston?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'reasoning_content': "Okay, the user is asking about the weather in Boston. I need to use the get_weather function. Let me check the function parameters. It requires a location, which in this case is Boston. So I'll call the function with location set to Boston. Make sure the JSON is correctly formatted with the name and arguments. No other parameters are needed. Just the location. Alright, that should do it.\n", 'tool_calls': [{'id': '6e41zbwdd', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 154, 'total_tokens': 260, 'completion_time': 0.190297964, 'completion_tokens_details': {'reasoning_tokens': 82}, 'prompt_time': 0.006027572, 'prompt_tokens_details': None, 'queue_time': 0.158901277, 'total_time': 0.196325536}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_2bfcc54d36', 'service_tier': 'on_demand', 'finish_reason':

### Tool Execution Loops

In [7]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

The weather in Boston is sunny ☀️. Let me know if you need more details!


In [8]:
messages

[{'role': 'user', 'content': "What's the weather in Boston?"},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Boston. I need to use the get_weather function. The function requires a location parameter. Boston is the location here. So I should call get_weather with location set to "Boston". Let me make sure there are no typos. Everything looks good. I\'ll format the tool call as specified.\n', 'tool_calls': [{'id': 'y5qds25fe', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 153, 'total_tokens': 247, 'completion_time': 0.145753126, 'completion_tokens_details': {'reasoning_tokens': 70}, 'prompt_time': 0.007024361, 'prompt_tokens_details': None, 'queue_time': 0.284070048, 'total_time': 0.152777487}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_d58dbe76cd', 'service_tier': 'on_demand', 'finis